# 1.Introduction

This Notebook tutorial will allow you to add firmware validation capability to your TrustFLEX (ATECC608A) device and test it with your PC

## 1.1.Firmware validation Transactions description

Secureboot is a new command on this device compared to earlier secure elements from Microchip. The feature assist the MCU to identify fraudulent code installed on it. When this command support is implemented, the MCU can send either the digest or signature or both to the ATECC608A. Then, the ATECC608A validates this information and responds to host either with Success/Fail or with MAC value.

<font color=red>This example is made to work only with TrustFLEX device. TrustFLEX devices come preconfigured but allow some further customization which this Tutorial will leverage. Refer to the TrustFLEX datasheet on www.microchip.com</font>

## 1.2.Tutorial Prerequisites:
    

The following code is runs on python 3.x environment. The folowing modules are also needed to run the example

1. cryptoauthlib
2. cryptography
3. ipywidgets

The next step of the tutorial with attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

In [ ]:
!python -m pip install -r ../../../requirements.txt

## 1.3. Load the necessary modules and helper functions

This Tutorial uses a number of python helper functions that are loaded by the following cell.
The below code does the following things:
1. Setup of cryptography module
2. Adds helper functions to get device name and device type

In [ ]:
import os, fnmatch, binascii
import ipywidgets as widgets
from ipywidgets import FileUpload
import cryptography
from cryptoauthlib import *
from common import *
from sboot_helper import *
from IPython.core.display import display
import hexrec.records as hr
import hexrec.blocks as hb
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec, utils
import warnings
warnings.filterwarnings('ignore')

# Setup cryptography 
crypto_be = cryptography.hazmat.backends.default_backend()
print("Importing modules - Successful")

# 2. Tutorial
Firmware validation support helps in avoiding the unauthorized firmware execution on a given system. The ability to securely upgrade the Microcontroller firmware starts with ensuring that the initial firmware has not been tampered with.
To have this functionality, once the firmware implementation is completed, it will be signed by the OEM signer to make the image authentic. On the product side, this application will be verified using OEM signers public key to ensure the authenticity. 

The notebook provides various steps involved to implement this using TrustFLEX device. Steps associated are,
1. Establish the communication between host (PC) and TrustFLEX
2. Generating a Key pair (optional - If resources are already generated, this step is not required)
    * a. Device provisioning:load the firmware validation public key into the TrustFLEX reserved slot
3. Sign the firmware
4. Update the firmware to product
5. Verify the firmware

## 2.1. Establish the communication between host (PC) and TrustFLEX
1. Initialize the library which also loads it
2. Check if the correct device is connected to the PC. If the wrong device is connected, a ValueError excpetion will be raised.

### 2.1.1. Select the CryptoAuthLib Hardware Abstraction layer to KIT USB HID

In [ ]:
ATCA_KIT_I2C_IFACE = 1
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6C #TFLXTLS device I2C address
ATCA_SUCCESS = 0
print("Set devtype as ECC608A - Successful")

### 2.1.2. Establish the connection with TrustFLEX device on the USB dongle and check its type
<font color=red>If the next Tutorial step yields an exception, the TrustFLEX is not correctly connected or of the wrong type.</font>

In [ ]:
# Initialize interface
assert atcab_init(cfg) == ATCA_SUCCESS, "Can't connect to the USB dongle"

# Get connected device type
info = bytearray(4)
assert atcab_info(info) == ATCA_SUCCESS, "Can't read the ECC608A device information"
dev_type = get_device_type_id(get_device_name(info))

# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    assert atcab_release() == ATCA_SUCCESS
    raise ValueError('Device not supported')
    
print("Device initialization - Successful")

## 2.2. Generating a firmware signing key pair

The most important step for firmware validation is having an ECC P256 key pair. The key pair can be generated using many of the options available like openssl (or) python modules etc..,. The private key will sign the image, the associated public key will verify the signed image.

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\Crypto Resource Generator.ipynb. </font> This key pair will be used for sign/verify operations in the steps to come. 

<font color = green>The keys are generated here using python module and is stored in computer's hard disk, this is not secure and is not recommended for actual production. In actual production environment these keys need to be stored in a secure storage as depitected below.</font>

<center><img src="img\firmware_validation_1.png" alt="**Get your Secure Elements here!**" /></center>

The following cell reads the firmware signing key pair generated and prints the associated public key

In [ ]:
key_file = '../../../TFLXTLS_resource_generation/slot_15_ecc_key_pair.pem'
with open(key_file, 'rb') as f:
    # Load the public key from key file
    priv_key = serialization.load_pem_private_key(
            data=f.read(),
            password=None,
            backend=default_backend())
    public_key = priv_key.public_key().public_numbers().encode_point()[1:]

print("Public Key:")
print(pretty_print_hex(public_key, indent='    '))

## 2.2.a Device Provisioning: Load the firmware signing public key into TrustFLEX device

The public key generated in [section 2.2](#2.2.-Generating-a-firmware-signing-key-pair) needs to be loaded into TrustFLEX device. This public key will be used to verify the signed image in the steps to follow. 

The TrustFLEX device comes preconfigured. The secure boot public key is expected to be loaded into the slot #15. Please refer to TrustFLEX datasheet on www.microchip.com website for detailed slot organization. 

In addition, the device is configured for FullDigest secure boot operation, which means after first successful secure boot, digest is stored and on subsequent boots, the signature verification will be skipped and the device will perform only a digest comparison. Therefore, only the firmware validation public key will need to be uploaded into TrustFLEX. After this step, you will end up with a system consisting of the following cryptographic assets

<font color=red>This step is already perfomed part of resource generation using TFLXTLS_resource_generation\Crypto Resource Generator.ipynb. </font>

<center><img src="img/firmware_validation_2.png" alt="**Get your Secure Elements here!**" /></center>

## 2.3. Sign the firmware

Once the target firmware development is complete, firmware needs to be secured by signing its digest with the private key generated in section [2.2](#2.2.-Generating-a-Firmware-Signing-key-pair). 

In this step the following things need to be done:
1. Get the firmware 
2. Hash the firmware with the SHA256 algorithm
3. sign the digest

<font color=purple>slot_15_ecc_key_pair.pem file created in section [2.2](#2.2.-Generating-a-firmware-signing-key-pair) is required to run the following code.</font>

The following block diagram provides the high level flow of signing operation on the OEM side.

<center><img src="img/firmware_validation_3.png" alt="**Get your Secure Elements here!**" /></center>

### Load firmware validation binary (HEX)

In [ ]:
firmvalid_img_object = FileUpload(accept='*.hex', multiple=False)
display(firmvalid_img_object)

### Load application binary (HEX)

In [ ]:
app_img_object = FileUpload(accept='*.hex', multiple=False)
display(app_img_object)

### Combining HEX files and apending signature

In [ ]:
combine_and_sign = widgets.Button(description = "Combine HEX")

def combine_hex(b):
    combine_sign_hex(firmvalid_img_object, app_img_object)
    
combine_and_sign.on_click(combine_hex)
display(combine_and_sign)

## 2.4 Update the firmware to product
This step upgrades the new firmware image digest to Product secure element. The digest and signature generated in the previous step will be provided to secure element using Secure boot command.  with option set to store (Full Copy) on successful validation of the digest and signature.

### Bus obfuscation
To ensure the digest is protected on the bus, its possible to obfuscate it while sending on the bus. <strong>IO Protection Key</strong> helps in encrypting the data.

On the TrustFLEX device Slot6 is configured to hold the IO protection key. This slot is configured to Never read and option to lock after writing the key. This is a unique key on the given setup. Every host and device pair agrees on a common key during first power up. Once the pair agrees on, this slot will be locked to avoid further updates. This ensures only this host and device pair knows the key. 

Any compromise of this key by the host, limits the impact to only this pair. All other host and device security remains intact.

<center><img src="img/firmware_validation_5.png" alt="**Get your Secure Elements here!**" /></center>

<strong>Note:</strong> A click button is provided at the end of the notebook to upgrade the image. This is done to easily access the upgrade and verify operations together.

## 2.5 Verify the application image

This step checks the application to execute is valid or not by issuing Secure boot command to secure element.

<center><img src="img/firmware_validation_4.png" alt="**Get your Secure Elements here!**" /></center>

Below code feeds required parameters for secureboot operation on TrustFLEX device. On receiving this, TrustFLEX performs digest store or comparison based on the mode.

In [ ]:
firmware_update = widgets.Button(description = "Firmware Update", tooltip = 'To update digest/signature using Secureboot command')
firmware_verify = widgets.Button(description = "Firmware Verify", tooltip = 'To Verify using Secureboot command')

io_prot_key = get_symmetric_key('../../../TFLXTLS_resource_generation/slot_6_secret_key.pem')

def secureboot_update(b):
# Generating a random number to use
    digest, signature = get_digest_signature()
    host_random = os.urandom(32)
    is_verified = AtcaReference(False)
    # In real application, this step should be carried ONLY on firmware upgrade
    print('Perform Application upgrade request... ')
    atcab_secureboot_mac(SECUREBOOT_MODE_FULL_COPY, digest, signature, host_random, io_prot_key, is_verified)
    if 1 == bool(is_verified.value):
        print('Secureboot Update Success...')
        firmware_update.button_style = 'success'
    else:
        firmware_update.button_style = 'danger'
        print('Secureboot Update failed...')

def secureboot_verify(b):
    # Generating a random number to use
    digest, signature = get_digest_signature()
    host_random = os.urandom(32)
    is_verified = AtcaReference(False)
    # Perform Secureboot operation on the application file
    print('Perform Application validation request... ')
    assert atcab_secureboot_mac(SECUREBOOT_MODE_FULL_STORE, digest, signature, host_random, io_prot_key, is_verified) == ATCA_SUCCESS
    if 1 == bool(is_verified.value):
        print('Secureboot Verify Success...')
        firmware_verify.button_style = 'success'
    else:
        firmware_verify.button_style = 'danger'
        print('Secureboot Verify failed...')

firmware_update.on_click(secureboot_update)
firmware_verify.on_click(secureboot_verify)
display(widgets.HBox((firmware_update, firmware_verify)))